In [ ]:
import os
import sys
from time import sleep
# Dataset augmentation types: 
# TPG: TimeStretch + PitchShift + GaussianSNR
# TPGB: TimeStretch + PitchShift + GaussianSNR + Background Noises
# TPGBIR: TimeStretch + PitchShift + GaussianSNR + Background Noises + Impulse Responses
aug_type = "TPGBIR"

model_state = "2025-11-12T14-29-42_0.0308"
model_choice = "openai_whisper_small"


model_base_path = os.path.join("model", model_choice)
model_state_path = os.path.join(model_base_path, model_state)

def get_latest_model_path(base_path):
    model_dirs = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    latest_model_dir = max(model_dirs, key=lambda d: os.path.getmtime(os.path.join(base_path, d)) if "checkpoint" not in d else 0)
    print(f"Latest model directory: {latest_model_dir}")
    return os.path.join(base_path, latest_model_dir)

# get_latest_model_dir(model_base_path)

i=0
while True:
    print(f"Running auto fine tune pipeline iteration {i}")
    %run preprocess.py --aug_type {aug_type} --src_dir "train" --label_file "train-toneless.csv" --dist_dir "train_{aug_type}" --n_augmentations 5
    %run preprocess.py --aug_type {aug_type} --src_dir "dict-sentence" --label_file "dict-sentence.csv" --dist_dir "dict-sentence_{aug_type}" --n_augmentations 1
    %run preprocess.py --aug_type {aug_type} --src_dir "dict-word" --label_file "dict-word" --dist_dir "dict-word_{aug_type}" --n_augmentations 1
    if i!=0:
        model_state_path = get_latest_model_path(model_base_path)
    
    %run train.py --dataset f"train_{aug_type}" --model_choice {model_choice} --model_state_path {model_state_path}
    model_state_path = get_latest_model_path(model_base_path)
    %run prediction.py --model_dir {model_state_path}

    %run train.py --dataset f"dict-sentence_{aug_type}" --model_choice {model_choice} --model_state_path {model_state_path}
    model_state_path = get_latest_model_path(model_base_path)
    %run prediction.py --model_dir {model_state_path}

    %run train.py --dataset f"train_{aug_type}" --model_choice {model_choice} --model_state_path {model_state_path}
    model_state_path = get_latest_model_path(model_base_path)
    %run prediction.py --model_dir {model_state_path}

    %run train.py --dataset f"dict-word_{aug_type}" --model_choice {model_choice} --model_state_path {model_state_path}
    model_state_path = get_latest_model_path(model_base_path)
    %run prediction.py --model_dir {model_state_path}

    i+=1
    sleep(5)


Running auto fine tune pipeline iteration 0
generating augmented data for type: TPGBIR
input path: ./data/train/train/train
input label file: ./data/train/train/train-toneless.csv
output path: ./data/train/train/ftrain_TPGBIR
output label file: ./data/train/train/ftrain_TPGBIR/metadata.csv
number of augmentations per file: 5


Processing files:   0%|          | 0/3119 [00:00<?, ?file/s]


FileNotFoundError: [Errno 2] No such file or directory: './data/train/train/ftrain_TPGBIR/34_aug1.wav'

generating augmented data for type: TPGBIR
input path: ./data/train/train/dict-sentence
input label file: ./data/train/train/dict-sentence.csv
output path: ./data/train/train/fdict-sentence_TPGBIR
output label file: ./data/train/train/fdict-sentence_TPGBIR/metadata.csv
number of augmentations per file: 1


Processing files:   0%|          | 0/17697 [00:00<?, ?file/s]


...

/home/user/workspace/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model state from: model/openai_whisper_small/2025-11-12T07-55-19_0.0282
Loading dataset from: data/train/train/ftrain_TPGBIR


FileNotFoundError: [Errno 2] No such file or directory: 'data/train/train/ftrain_TPGBIR/metadata.csv'

Latest model directory: 2025-11-12T14-29-42_0.0308
Extracted 1259 unique words from CSV.
Extracted 832 unique words from lexicon.
Unique words count: 1304
Loading model from: model/openai_whisper_small/2025-11-12T14-29-42_0.0308


Processing audio:   0%|          | 0/346 [00:00<?, ?file/s]`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these values explicitly.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> to see related `.generate()` flags.
A custom logits processor 